# Structured Planning Agent

A key pattern in agents is the ability to plan. ReAct for example, uses a structured approach to decompose an input into a set of function calls and thoughts, in order to reason about a final response.

However, breaking down the initial input/task into several sub-tasks can make the ReAct loop (or other reasoning loops) easier to execute.

The `StructuredPlanningAgnet` in LlamaIndex wraps any agent worker (ReAct, Function Calling, Chain-of-Abstraction, etc.) and decomposes an initial input into several sub-tasks. Each sub-task is represented by an input, expected outcome, and any dependendant sub-tasks that should be completed first.

This notebook walks through both the high-level and low-level usage of this agent.

**NOTE:** This agent leverages both structured outputs and agentic reasoning. Because of this, we would recommend a capable LLM (OpenAI, Anthropic, etc.), and open-source LLMs may struggle to plan without prompt engineering or fine-tuning.

## Setup

In order to create plans, we need a set of tools to create plans on top of. Here, we use some classic 10k examples.

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Use ollama in JSON mode
Settings.llm = OpenAI(
    model="gpt-4-turbo",
    temperature=0.1,
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

# Load documents, create tools
lyft_documents = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_documents = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

lyft_index = VectorStoreIndex.from_documents(lyft_documents)
uber_index = VectorStoreIndex.from_documents(uber_documents)

lyft_tool = QueryEngineTool.from_defaults(
    lyft_index.as_query_engine(),
    name="lyft_2021",
    description="Useful for asking questions about Lyft's 2021 10-K filling.",
)

uber_tool = QueryEngineTool.from_defaults(
    uber_index.as_query_engine(),
    name="uber_2021",
    description="Useful for asking questions about Uber's 2021 10-K filling.",
)

## High Level API

In this section, we cover the high-level API for creating with and chatting with a structured planning agent.

### Create the Agent

In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### Give the agent a complex task

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
response = agent.chat(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
Summarize the key risk factors from Lyft's 2021 10-K filing. -> A summary of the key risk factors for Lyft as outlined in their 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
Summarize the key risk factors from Uber's 2021 10-K filing. -> A summary of the key risk factors for Uber as outlined in their 2021 10-K filing.
deps: []


Combine Risk Factors Summaries:
Combine the summaries of key risk factors for Lyft and Uber from their 2021 10-K filings into a comprehensive overview. -> A comprehensive summary of the key risk factors for both Lyft and Uber as outlined in their respective 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


> Running step a1428164-9f35-4a0e-9d6d-3328ebe490d4. Step input: Summarize the key risk factors from Lyft's 2021 10-K filing.
Added user message to memory: Summarize the key risk factors from Lyft's 2021 10-K filing.
> Running step 16a0d1f5-03ba-4b47-af0f-ec8758e6dda

In [ ]:
print(str(response))

assistant: The combined key risk factors from the 2021 10-K filings of Lyft and Uber highlight several shared and distinct challenges in the ridesharing and mobility industry:

1. **Pandemic and Economic Impact**: Both companies are significantly affected by the COVID-19 pandemic and broader economic conditions, which influence market demand and operational stability.

2. **Regulatory and Labor Issues**: A major concern is the classification of drivers as independent contractors versus employees, impacting labor costs and operational flexibility.

3. **Intense Competition**: They face fierce competition in the mobility, delivery, and logistics sectors, characterized by low entry barriers and aggressive, well-funded competitors.

4. **Operational and Financial Uncertainty**: Both companies deal with financial unpredictability and operational challenges, including managing insurance, claims, and maintaining profitability amidst high expenses.

5. **Technological and Service Evolution**: 

## Changing Prompts

The `StructuredPlanningAgent` has two key prompts:
1. The initial planning prompt
2. The plan refinement prompt

Below, we show how to configure these prompts, using the defaults as an example.

In [ ]:
DEFAULT_INITIAL_PLAN_PROMPT = """\
Think step-by-step. Given a task and a set of tools, create a comprehesive, end-to-end plan to accomplish the task.
Keep in mind not every task needs to be decomposed into multiple sub-tasks if it is simple enough.
The plan should end with a sub-task that satisfies the overall task.

The tools available are:
{tools_str}

Overall Task: {task}
"""

DEFAULT_PLAN_REFINE_PROMPT = """\
Think step-by-step. Given an overall task, a set of tools, and completed sub-tasks, update (if needed) the remaining sub-tasks so that the overall task can still be completed.
The plan should end with a sub-task that satisfies the overall task.
If the remaining sub-tasks are sufficient, you can skip this step.

The tools available are:
{tools_str}

Overall Task:
{task}

Completed Sub-Tasks + Outputs:
{completed_outputs}

Remaining Sub-Tasks:
{remaining_sub_tasks}
"""

In [ ]:
agent = StructuredPlannerAgent(
    worker,
    tools=[lyft_tool, uber_tool],
    initial_plan_prompt=DEFAULT_INITIAL_PLAN_PROMPT,
    plan_refine_prompt=DEFAULT_PLAN_REFINE_PROMPT,
    verbose=True,
)

## Low-level API [Advanced] 

In this section, we use the same agent, but expose the lower-level steps that are happening under the hood.

This is useful for when you want to expose the underlying plan, tasks, etc. to a human to modify them on the fly, or for debugging and running things step-by-step.

### Create the Agent

In [ ]:
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)

# create the react worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

### Create the initial tasks and plan

In [ ]:
plan_id = agent.create_plan(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Extract Lyft Risk Factors:
Extract the key risk factors from Lyft's 2021 10-K filing. -> A detailed list of key risk factors for Lyft from its 2021 10-K filing.
deps: []


Extract Uber Risk Factors:
Extract the key risk factors from Uber's 2021 10-K filing. -> A detailed list of key risk factors for Uber from its 2021 10-K filing.
deps: []


Summarize Risk Factors:
Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings. -> A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




### Inspect the initial tasks and plan

In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A detailed list of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A detailed list of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### Execute the first set of tasks

Here, we execute the first set of tasks with their dependencies met.

In [ ]:
next_tasks = agent.state.get_next_sub_tasks(plan_id)

for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)


for sub_task in next_tasks:
    response = agent.run_task(sub_task.name)
    agent.mark_task_complete(plan_id, sub_task.name)

===== Sub Task Extract Lyft Risk Factors =====
Expected output:  A detailed list of key risk factors for Lyft from its 2021 10-K filing.
Dependencies:  []
===== Sub Task Extract Uber Risk Factors =====
Expected output:  A detailed list of key risk factors for Uber from its 2021 10-K filing.
Dependencies:  []
> Running step 4f708bcd-3078-4138-897b-e7f643fc7f35. Step input: Extract the key risk factors from Lyft's 2021 10-K filing.
Added user message to memory: Extract the key risk factors from Lyft's 2021 10-K filing.
=== Calling Function ===
Calling function: lyft_2021 with args: {"input": "key risk factors"}
=== Function Output ===
Key risk factors for Lyft include general economic factors such as the impact of the COVID-19 pandemic, natural disasters, and macroeconomic conditions; operational factors such as limited operating history, competition, unpredictability of results, and the ability to attract and retain drivers and riders; and specific risks related to technology, such as a

If we wanted to, we could even execute each task in a step-wise fashion. It would look something like this:

```python
# Step-wise execution per task

for sub_task in next_tasks:
    # get the task from the state 
    task = agent.state.get_task(sub_task.name)

    # run intial resoning step
    step_output = agent.run_step(task.task_id)

    # loop until the last step is reached
    while not step_output.is_last:
        step_output = agent.run_step(task.task_id)
    
    # finalize the response and commit to memory
    agent.finalize_response(task.task_id, step_output=step_output)
```

### Check if we are done

If there are no remaining tasks, then we can stop. Otherwise, we can refine the current plan and continue

In [ ]:
next_tasks = agent.get_next_tasks(plan_id)
print(len(next_tasks))

1


In [ ]:
for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task} =====")

===== Sub Task Summarize Risk Factors =====


### Refine the plan

Since we have tasks remaining, lets refine our plan to make sure we are on track.

In [ ]:
# refine the plan
agent.refine_plan(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings.",
    plan_id,
)

=== Refined plan ===
Summarize Risk Factors:
Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings. -> A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
deps: ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']




In [ ]:
plan = agent.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Summarize Risk Factors =====
Expected output:  A comprehensive summary of the key risk factors for Lyft and Uber from their 2021 10-K filings.
Dependencies:  ['Extract Lyft Risk Factors', 'Extract Uber Risk Factors']


### Loop until done

With our plan refined, we can repeat this process until we have no more tasks to run.

In [ ]:
import asyncio

while True:
    # are we done?
    next_tasks = agent.get_next_tasks(plan_id)
    if len(next_tasks) == 0:
        break

    # run concurrently for better performance
    responses = await asyncio.gather(
        *[agent.arun_task(task_id) for task_id in next_tasks]
    )
    for task_id in next_tasks:
        agent.mark_task_complete(plan_id, task_id)

    # refine the plan
    await agent.arefine_plan(
        "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings.",
        plan_id,
    )

> Running step 70356e9a-98ee-49f5-b15f-e5a6b43381d0. Step input: Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings.
Added user message to memory: Summarize the key risk factors for both Lyft and Uber based on the extracted information from their 2021 10-K filings.
=== LLM Response ===
The key risk factors for Lyft and Uber from their 2021 10-K filings highlight several areas of concern for both companies, with some overlapping and unique challenges:

### Common Risk Factors:
- **Economic and Market Conditions:** Both companies are affected by general economic factors such as macroeconomic conditions and the impact of the COVID-19 pandemic. Additionally, Uber faces specific market risks like interest rate risk, investment risk, and foreign currency risk due to its global operations.

### Lyft-Specific Risk Factors:
- **Operational Challenges:** Lyft faces risks related to its limited operating history, competition, unpr

By the end, we should have a single response, which is our final response

In [ ]:
print(str(responses[-1]))

assistant: The key risk factors for Lyft and Uber from their 2021 10-K filings highlight several areas of concern for both companies, with some overlapping and unique challenges:

### Common Risk Factors:
- **Economic and Market Conditions:** Both companies are affected by general economic factors such as macroeconomic conditions and the impact of the COVID-19 pandemic. Additionally, Uber faces specific market risks like interest rate risk, investment risk, and foreign currency risk due to its global operations.

### Lyft-Specific Risk Factors:
- **Operational Challenges:** Lyft faces risks related to its limited operating history, competition, unpredictability of results, and the ability to attract and retain drivers and riders.
- **Technology and Insurance Risks:** Challenges include autonomous vehicle development, security breaches, reliance on third-party service providers, insurance coverage adequacy, and handling of auto-related insurance claims by third parties.
- **Regulatory a